# PDMS 3D model contextualization
This notebook contains a workflow for contextualizing 3D models made in PDMS which typically works well for oil and gas 3D models. 

Authors: Alina Astrakova and Anders Hafreager

In [ ]:
import os
import json

import numpy as np
import pandas as pd

from getpass import getpass
from cognite.client import CogniteClient
from cognite.client.data_classes.three_d import ThreeDAssetMapping

# Initialize

In [ ]:
project = "akerbp"
api_key = getpass()

In [ ]:
client = CogniteClient(api_key, project, "dshub")

In [ ]:
client.three_d.models.list()

In [ ]:
client.three_d.revisions.list(2974710125783473, published=True) # If you don't find the revision, try published=False

In [ ]:
client.assets.list(root=True)

In [ ]:
# Define 3d model_id and revision
model_id = 2974710125783473
revision_id = 8462111230533116

# Define root_id for assets
root_id = 4175743261975966

# Download data

In [ ]:
# Download 3D nodes. This may take a while ...
threed_nodes = client.three_d.revisions.list_nodes(model_id=model_id, revision_id=revision_id, limit=-1)

In [ ]:
# The 3D node hierarchy is often made of nodes with names on the form "/21PT1019"
# with children nodes with names "BRANCH 1 of /21PT1019".
# We only want to map the parent node, so remove all nodes with such names.
filtered_nodes = threed_nodes.dump()
filtered_nodes = list(filter(lambda x: x["name"] != "", nodes_list))
print("%d non empty node names" % len(filtered_nodes))
filtered_nodes = list(filter(lambda x: "EQUIPMENT" not in x["name"], filtered_nodes))
print("%d node names without EQUIPMENT" % len(filtered_nodes))
filtered_nodes = list(filter(lambda x: "BRANCH" not in x["name"], filtered_nodes))
print("%d node names without BRANCH" % len(filtered_nodes))
filtered_nodes = list(filter(lambda x: "STRUCTURE" not in x["name"], filtered_nodes))
print("%d node names without STRUCTURE" % len(filtered_nodes))
filtered_nodes = list(filter(lambda x: " OF " not in x["name"], filtered_nodes))
print("%d node names without OF " % len(filtered_nodes))
filtered_nodes = list(filter(lambda x: " of " not in x["name"], filtered_nodes))
print("%d node names without of " % len(filtered_nodes))
filtered_nodes = list(filter(lambda x: "Box" not in x["name"], filtered_nodes))
print("%d node names without Box " % len(filtered_nodes))
filtered_nodes = list(filter(lambda x: "Cylinder" not in x["name"], filtered_nodes))
print("%d node names without Cylinder " % len(filtered_nodes))
filtered_nodes = list(filter(lambda x: "Facet Group" not in x["name"], filtered_nodes))
print("%d node names without Facet Group " % len(filtered_nodes))
filtered_nodes = list(filter(lambda x: "curve" not in x["name"], filtered_nodes))
print("%d node names without curve " % len(filtered_nodes))
filtered_nodes = list(filter(lambda x: "Pyramid" not in x["name"], filtered_nodes))
print("%d node names without Pyramid " % len(filtered_nodes))
filtered_nodes = list(filter(lambda x: "Line" not in x["name"], filtered_nodes))
print("%d node names without Line " % len(filtered_nodes))
filtered_node_names = list(map(lambda x: x["name"], filtered_nodes))

In [ ]:
# Download assets
df_assets = client.assets.retrieve_subtree(root_id)
asset_names = list(map(lambda x: x["name"], df_assets.dump()))

# Run the entity matcher

In [ ]:
from cognite.datastudio.entity_matcher import EntityMatcher
entity_matcher = EntityMatcher(client)
model = entity_matcher.fit(asset_names)
matches = model.predict(filtered_node_names)

In [ ]:
# This may require some work of verification. Run 
threshold = 0.9
good_matches = list(filter(lambda x: x["score"] != None and x["score"] > threshold, matches))
print("Got %d matches with score > %f" % (len(good_matches), threshold))
good_matches = pd.DataFrame.from_records(good_matches)
good_matches

In [ ]:
# Create map from name to object for assets and 3D nodes
assets_by_name = {}
for asset in df_assets.dump():
    assets_by_name[asset["name"]] = asset

threed_nodes_by_name = {}
for node in threed_nodes.dump():
    threed_nodes_by_name[node["name"]] = node

# Create asset mappings

In [ ]:
asset_mappings = []
for match in good_matches.to_records():
    asset_id = assets_by_name[match["predicted"]]["id"]
    node_id = threed_nodes_by_name[match["input"]]["id"]
    asset_mappings.append(ThreeDAssetMapping(node_id=node_id, asset_id=asset_id))

In [ ]:
# Write mappings to CDF
res = client.three_d.asset_mappings.create(model_id=model_id, revision_id=revision_id, asset_mapping=asset_mappings)